In [1]:
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.types import *
import pandas as pd
import pyspark.sql.functions as f 

spark = SparkSession.builder \
        .appName("Spark Benchmarking") \
        .master('local[16]') \
        .getOrCreate()


In [3]:
def spark_run(file_name, spark):
    SCHEMA = StructType([StructField('key', IntegerType(), False),
                         StructField('value1', IntegerType(), False),
                         StructField('value2', IntegerType(), False),
                         StructField('value3', IntegerType(), False),])

    file_vals = spark\
        .read\
        .load(file_name, format="csv", header='true', schema=SCHEMA)\
        .groupby("key")\
        .agg(f.mean((f.col("value1")-f.col("value2"))/f.col("value3")).alias("mean"))
    
    fl = (file_name.split('.')[1]).split('/')[2]
    file_vals.write.format('parquet').mode("overwrite").save(f"./output/{fl}")
    del file_vals
    

In [ ]:

import glob
for filename in glob.glob('./data/*.csv'):
    print(filename)
    %timeit spark_run(filename, spark)

In [5]:
spark_run('./data/sample_file_8.csv', spark)